- cancel pytorch mlp and import pytorch tabnet

In [1]:
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet

Looking in links: /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl
Processing /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl


In [2]:
import sys
!cp ../input/rapids/rapids.0.15.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [3]:
import os
import warnings
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from tqdm import tqdm_notebook as tqdm
from category_encoders import CountEncoder
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import KFold, StratifiedKFold
sys.path.append('../input/multilabelstraifier/')
from ml_stratifiers import MultilabelStratifiedKFold
warnings.filterwarnings('ignore')

import time
import torch
import random
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf

from sklearn.kernel_approximation import Nystroem
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
from sklearn.kernel_ridge import KernelRidge
from cuml.svm import SVC, SVR

from tensorflow.keras import layers,regularizers,Sequential,backend,callbacks,optimizers,metrics,losses
import tensorflow_addons as tfa
from keras import backend as K

from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

# Preprocess & Feature engineering

In [4]:
DATA_DIR = '/kaggle/input/lish-moa/'
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
test = pd.read_csv(DATA_DIR + 'test_features.csv')
#non_targets = pd.read_csv(DATA_DIR + 'train_targets_nonscored.csv')

In [5]:
#non_target_feats = [i for i in non_targets.columns if i != "sig_id"]
#nontarget_dists = pd.DataFrame(np.sum(non_targets[non_target_feats])).reset_index(drop=False)
#nontarget_dists.columns = ["target", "number"]
#nontarget_dists = nontarget_dists.sort_values("number", ascending=False).reset_index(drop=True)
#drop_list1 = list(nontarget_dists[nontarget_dists.number==0]["target"].values)
#non_targets.drop(drop_list1, axis=1, inplace=True)
#print("shape after 1st drop:", non_targets.shape)
#non_target_feats = [i for i in non_targets.columns if i != "sig_id"]
#print(len(non_target_feats))

In [6]:
target_feats = [ i for i in targets.columns if i != "sig_id"]
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

In [7]:
noncons_train_index = train[train.cp_type=="ctl_vehicle"].index
cons_train_index = train[train.cp_type!="ctl_vehicle"].index
noncons_test_index = test[test.cp_type=="ctl_vehicle"].index
cons_test_index = test[test.cp_type!="ctl_vehicle"].index

test = test[test.index.isin(cons_test_index)].reset_index(drop=True)
train = train[train.index.isin(cons_train_index)].reset_index(drop=True)
fn_targets = targets.drop("sig_id", axis=1).copy()
fn_targets = fn_targets[fn_targets.index.isin(cons_train_index)].copy().reset_index(drop=True).to_numpy()
#fn_non_targets = non_targets.drop("sig_id", axis=1).copy()
#fn_non_targets = fn_non_targets[fn_non_targets.index.isin(cons_train_index)].copy().reset_index(drop=True).to_numpy()
y = targets.drop("sig_id", axis=1).copy()

In [8]:
def fe_new(df, remove_features):
    tmp = df.copy()
    tmp['g_sum'] = tmp[g_feats].sum(axis = 1)
    tmp['g_mean'] = tmp[g_feats].mean(axis = 1)
    tmp['g_std'] = tmp[g_feats].std(axis = 1)
    tmp['g_kurt'] = tmp[g_feats].kurtosis(axis = 1)
    tmp['g_skew'] = tmp[g_feats].skew(axis = 1)
    tmp['c_sum'] = tmp[c_feats].sum(axis = 1)
    tmp['c_mean'] = tmp[c_feats].mean(axis = 1)
    tmp['c_std'] = tmp[c_feats].std(axis = 1)
    tmp['c_kurt'] = tmp[c_feats].kurtosis(axis = 1)
    tmp['c_skew'] = tmp[c_feats].skew(axis = 1)
    tmp['gc_sum'] = tmp[c_feats + g_feats].sum(axis = 1)
    tmp['gc_mean'] = tmp[c_feats + g_feats].mean(axis = 1)
    tmp['gc_std'] = tmp[c_feats + g_feats].std(axis = 1)
    tmp['gc_kurt'] = tmp[c_feats + g_feats].kurtosis(axis = 1)
    tmp['gc_skew'] = tmp[c_feats + g_feats].skew(axis = 1)
    tmp.loc[:, 'cp_dose'] = tmp.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    tmp.drop(remove_features, axis=1, inplace=True)
    return tmp

def fe_simple(df, remove_features):
    tmp = df.copy()
    tmp.drop(remove_features, axis=1, inplace=True)
    tmp.loc[:, 'cp_dose'] = tmp.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    return tmp

remove_features = ["cp_type" , "sig_id"]
        
train_pl = fe_new(train, remove_features)
test_pl = fe_new(test, remove_features)

train = fe_simple(train, remove_features)
test = fe_simple(test, remove_features)
    
print(train_pl.shape, test_pl.shape)
print(train.shape, test.shape)

(21948, 889) (3624, 889)
(21948, 874) (3624, 874)


# data preparation

In [9]:
# xgb --------------------------
X = train_pl.copy()
X_test = test_pl.copy()

# logistic regression and svm-----------------------
fn_train = train.copy() 
fn_test = test.copy() 

ss = preprocessing.StandardScaler()
fn_train= ss.fit_transform(fn_train)
fn_test = ss.transform(fn_test)

# pytorch ----------------------------------
py_train = train.copy()
py_test = test.copy()

ss = preprocessing.RobustScaler()
py_train= ss.fit_transform(py_train)
py_test = ss.transform(py_test)

# 1st XGB

In [10]:
np.random.seed(42)
NFOLDS=5

classifier = MultiOutputClassifier(XGBClassifier(tree_method='gpu_hist'))

clf = Pipeline([('classify', classifier)
               ])

params = {'classify__estimator__gamma': 3.6975,
          'classify__estimator__learning_rate': 0.0803,
          'classify__estimator__max_delta_step': 2.0706,
          'classify__estimator__max_depth': 10,
          'classify__estimator__min_child_weight': 31.5800,
          'classify__estimator__n_estimators': 166,
         }

clf.set_params(**params)

Pipeline(steps=[('classify',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               gamma=3.6975,
                                                               gpu_id=None,
                                                               importance_type='gain',
                                                               interaction_constraints=None,
                                                               learning_rate=0.0803,
                                                               max_delta_step=2.0706,
                                                   

In [11]:
def modelling_xgb(X, y, X_test, seed):
    oof_preds = np.zeros(y.shape)
    test_preds = np.zeros((X_test.shape[0], y.shape[1]))
    oof_losses = []
    mskf = MultilabelStratifiedKFold(n_splits=NFOLDS, random_state=seed, shuffle=True)
    for fn, (trn_idx, val_idx) in enumerate(mskf.split(X, y)):
        print('Starting fold: ', fn)
        X_train, X_val = X.iloc[trn_idx,:].to_numpy(), X.iloc[val_idx,:].to_numpy()
        y_train, y_val = y[trn_idx], y[val_idx]
    
        clf.fit(X_train, y_train)
        val_preds = clf.predict_proba(X_val) # list of preds per class
        val_preds = np.array(val_preds)[:,:,1].T # take the positive class
        oof_preds[val_idx] = val_preds
    
        loss = log_loss(np.ravel(y_val), np.ravel(val_preds))
        print(loss)
        oof_losses.append(loss)
        preds = clf.predict_proba(X_test)
        preds = np.array(preds)[:,:,1].T # take the positive class
        test_preds += preds / NFOLDS
    
    print(oof_losses)
    print('Mean OOF loss across folds', np.mean(oof_losses))
    print('STD OOF loss across folds', np.std(oof_losses))
    return oof_preds, test_preds

In [12]:
seeds = [42]
xgb1_oof = np.zeros(fn_targets.shape)
xgb1_test = np.zeros((test.shape[0], fn_targets.shape[1]))
for seed_ in seeds:
    ind_preds, ind_test_preds = modelling_xgb(X, fn_targets, X_test, seed_)
    xgb1_oof += ind_preds / len(seeds)
    xgb1_test += ind_test_preds / len(seeds)

Starting fold:  0
0.017890018527333493
Starting fold:  1
0.017878259676696682
Starting fold:  2
0.017561236619825567
Starting fold:  3
0.017891451323207592
Starting fold:  4
0.017919797908421412
[0.017890018527333493, 0.017878259676696682, 0.017561236619825567, 0.017891451323207592, 0.017919797908421412]
Mean OOF loss across folds 0.017828152811096946
STD OOF loss across folds 0.00013415501741458868


In [13]:
check_xgb1 = np.zeros([targets.shape[0], targets.shape[1]-1])
check_xgb1[cons_train_index,:] =  xgb1_oof
print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(check_xgb1)))

OOF log loss:  0.016431194422447486


In [14]:
del X_test
del X

# 1st pytorch

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
batch_size = 128
train_epochs = 40
n_folds=5

def mean_log_loss(y_true, y_pred):
    metrics = []
    for i, target in enumerate(target_feats):
        metrics.append(log_loss(y_true[:, i], y_pred[:, i].astype(float), labels=[0,1]))
    return np.mean(metrics)

def seed_everything(seed=1234): 
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
class MoaModel(nn.Module):
    def __init__(self, num_columns, last_columns_num):
        super(MoaModel, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_columns)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_columns, 2048))
        
        self.batch_norm2 = nn.BatchNorm1d(2048)
        self.dropout2 = nn.Dropout(0.6)
        self.dense2 = nn.utils.weight_norm(nn.Linear(2048, 1048))
        
        self.batch_norm3 = nn.BatchNorm1d(1048)
        self.dropout3 = nn.Dropout(0.6)
        self.dense3 = nn.utils.weight_norm(nn.Linear(1048, last_columns_num))
        
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x
    
def modelling_torch(tr, target, te, sample_seed, init_num, last_num, layer):
    seed_everything(seed=sample_seed) 
    X_train = tr.copy()
    y_train = target.copy()
    X_test = te.copy()
    test_len = X_test.shape[0]

    mskf=MultilabelStratifiedKFold(n_splits = n_folds, shuffle=True, random_state=2)
    models = []
    
    X_test = torch.tensor(X_test, dtype=torch.float32)
    X_test = torch.utils.data.TensorDataset(X_test) 
    test_loader = torch.utils.data.DataLoader(X_test, batch_size=batch_size, shuffle=False)
    
    oof = np.zeros([len(X_train),y_train.shape[1]])
    oof_targets = np.zeros([len(X_train),y_train.shape[1]])
    pred_value = np.zeros([test_len, y_train.shape[1]])
    scores = []
    for fold, (train_index, valid_index) in enumerate(mskf.split(X_train, y_train)):
        print("Fold "+str(fold+1))
        X_train2 = torch.tensor(X_train[train_index,:], dtype=torch.float32)
        y_train2 = torch.tensor(y_train[train_index], dtype=torch.float32)

        X_valid2 = torch.tensor(X_train[valid_index,:], dtype=torch.float32)
        y_valid2 = torch.tensor(y_train[valid_index], dtype=torch.float32)
            
        clf = MoaModel(init_num, last_num)
        loss_fn = torch.nn.BCEWithLogitsLoss() 
        optimizer = optim.Adam(clf.parameters(), lr = 0.001, weight_decay=1e-5) 
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, eps=1e-4, verbose=True)
        
        train = torch.utils.data.TensorDataset(X_train2, y_train2)
        valid = torch.utils.data.TensorDataset(X_valid2, y_valid2)
        
        clf.to(device)
        
        train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True) 
        valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
        
        best_val_loss = np.inf
        for epoch in range(train_epochs):
            start_time = time.time()
            clf.train()
            avg_loss = 0.
            for x_batch, y_batch in tqdm(train_loader, disable=True):
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                y_pred = clf(x_batch) 
                loss = loss_fn(y_pred, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                avg_loss += loss.item() / len(train_loader) 
    
            clf.eval()
            avg_val_loss = 0.
            for i, (x_batch, y_batch) in enumerate(valid_loader): 
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                y_pred = clf(x_batch).detach()
                avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)
        
            elapsed_time = time.time() - start_time 
            scheduler.step(avg_val_loss)

            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                print('Best model: Epoch {} \t loss={:.6f} \t val_loss={:.6f} \t time={:.2f}s'.format(
                    epoch + 1, avg_loss, avg_val_loss, elapsed_time))
                torch.save(clf.state_dict(), 'best-model-parameters.pt')
         
        pred_model = MoaModel(init_num, last_num)
        pred_model.load_state_dict(torch.load('best-model-parameters.pt'))
        pred_model.eval()
        
        # validation check ----------------
        oof_epoch = np.zeros([X_valid2.size(0), y_train.shape[1]])
        target_epoch = np.zeros([X_valid2.size(0), y_train.shape[1]])
        for i, (x_batch, y_batch) in enumerate(valid_loader): 
                y_pred = pred_model(x_batch).sigmoid().detach()
                oof_epoch[i * batch_size:(i+1) * batch_size,:] = y_pred.cpu().numpy()
                target_epoch[i * batch_size:(i+1) * batch_size,:] = y_batch.cpu().numpy()
        print("Fold {} log loss: {}".format(fold+1, mean_log_loss(target_epoch, oof_epoch)))
        scores.append(mean_log_loss(target_epoch, oof_epoch))
        oof[valid_index,:] = oof_epoch
        oof_targets[valid_index,:] = target_epoch
        #-----------------------------------
        
        # test predcition --------------
        test_preds = np.zeros([test_len, y_train.shape[1]])
        for i, (x_batch,) in enumerate(test_loader): 
            y_pred = pred_model(x_batch).sigmoid().detach()
            test_preds[i * batch_size:(i+1) * batch_size, :] = y_pred.cpu().numpy()
        pred_value += test_preds / n_folds
        # ------------------------------
        
    print("Seed {}".format(seed_))
    for i, ele in enumerate(scores):
        print("Fold {} log loss: {}".format(i+1, scores[i]))
    print("Std of log loss: {}".format(np.std(scores)))
    print("Total log loss: {}".format(mean_log_loss(oof_targets, oof)))
    
    return oof, oof_targets, pred_value

cuda


In [16]:
#seeds = [10,40]
#pytorch1_oof = np.zeros([len(py_train),fn_targets.shape[1]])
#pytorch1_test = np.zeros([len(py_test),fn_targets.shape[1]])

#for seed_ in seeds:
#    oof, oof_targets, pytorch_pred = modelling_torch(py_train, fn_targets, py_test, seed_, py_train.shape[1], fn_targets.shape[1],1)
#    pytorch1_oof += oof / len(seeds)
#    pytorch1_test += pytorch_pred / len(seeds)

In [17]:
#check_pytorch1 = np.zeros([targets.shape[0], targets.shape[1]-1])
#check_pytorch1[cons_train_index,:] = pytorch1_oof
#print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(check_pytorch1)))

# 1st tabnet

In [18]:
class LogitsLogLoss(Metric):
    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        """
        Compute LogLoss of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            LogLoss of predictions vs targets.
        """
        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1-y_true)*np.log(1-logits+1e-15) + y_true*np.log(logits+1e-15)
        return np.mean(-aux)

In [19]:
MAX_EPOCH=200

def seed_tabnet_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
def modelling_tabnet(tr, target, te, sample_seed):
    seed_tabnet_everything(sample_seed) 
    tabnet_params = dict(n_d=12, n_a=12, n_steps=1, gamma=1.3, seed = sample_seed,
                     lambda_sparse=0, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type='entmax',
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=10,
                     )
    test_cv_preds = []
    
    NB_SPLITS = 5
    mskf = MultilabelStratifiedKFold(n_splits=NB_SPLITS, random_state=0, shuffle=True)
    oof_preds = np.zeros([len(tr),target.shape[1]])
    scores = []
    for fold_nb, (train_idx, val_idx) in enumerate(mskf.split(train, target)):
        print("FOLDS : ", fold_nb+1)

        ## model
        X_train, y_train = tr[train_idx, :], target[train_idx, :]
        X_val, y_val = tr[val_idx, :], target[val_idx, :]
        model = TabNetRegressor(**tabnet_params)
        
        model.fit(X_train=X_train,
              y_train=y_train,
              eval_set=[(X_val, y_val)],
              eval_name = ["val"],
              eval_metric = ["logits_ll"],
              max_epochs=MAX_EPOCH,
              patience=20, batch_size=1024, virtual_batch_size=128,
              num_workers=1, drop_last=False,
              # use binary cross entropy as this is not a regression problem
              loss_fn=torch.nn.functional.binary_cross_entropy_with_logits)
    
        preds_val = model.predict(X_val)
        preds =  1 / (1 + np.exp(-preds_val))
        score = np.min(model.history["val_logits_ll"])
        oof_preds[val_idx,:] = preds
        scores.append(score)
        
        # preds on test
        preds_test = model.predict(te)
        test_cv_preds.append(1 / (1 + np.exp(-preds_test)))

    test_preds_all = np.stack(test_cv_preds)
    return oof_preds, test_preds_all

In [20]:
tabnet1_oof = np.zeros([len(py_train),fn_targets.shape[1]])
tabnet1_test = np.zeros([len(py_test),fn_targets.shape[1]])

seeds = [0,1]
for seed_ in seeds:
    oof_preds, test_preds_all = modelling_tabnet(py_train, fn_targets, py_test, seed_)
    tabnet1_oof += oof_preds / len(seeds)
    tabnet1_test += test_preds_all.mean(axis=0) / len(seeds)

FOLDS :  1
Device used : cuda
epoch 0  | loss: 0.56805 | val_logits_ll: 0.30547 |  0:00:01s
epoch 10 | loss: 0.02066 | val_logits_ll: 0.02053 |  0:00:11s
epoch 20 | loss: 0.01902 | val_logits_ll: 0.01885 |  0:00:21s
epoch 30 | loss: 0.01782 | val_logits_ll: 0.01842 |  0:00:32s
epoch 40 | loss: 0.01723 | val_logits_ll: 0.01774 |  0:00:42s
epoch 50 | loss: 0.01699 | val_logits_ll: 0.01863 |  0:00:52s
epoch 60 | loss: 0.01669 | val_logits_ll: 0.01722 |  0:01:04s
epoch 70 | loss: 0.0167  | val_logits_ll: 0.0171  |  0:01:15s
epoch 80 | loss: 0.01643 | val_logits_ll: 0.01703 |  0:01:25s
epoch 90 | loss: 0.01611 | val_logits_ll: 0.01693 |  0:01:35s
epoch 100| loss: 0.01599 | val_logits_ll: 0.01745 |  0:01:45s

Early stopping occured at epoch 104 with best_epoch = 84 and best_val_logits_ll = 0.01675
Best weights from best epoch are automatically used!
FOLDS :  2
Device used : cuda
epoch 0  | loss: 0.56657 | val_logits_ll: 0.30328 |  0:00:00s
epoch 10 | loss: 0.02066 | val_logits_ll: 0.02041 | 

In [21]:
check_tabnet1 = np.zeros([targets.shape[0], targets.shape[1]-1])
check_tabnet1[cons_train_index,:] = tabnet1_oof
print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(check_tabnet1)))

OOF log loss:  0.015328942805464933


# 1st SVM

In [22]:
N_STARTS = 3
N_SPLITS = 5

svm0_oof = np.zeros([len(fn_train), fn_targets.shape[1]])
svm0_test = np.zeros([len(fn_test), fn_targets.shape[1]])

for ind in tqdm(range(len(target_feats))):

    ind_target_sum = targets.drop("sig_id", axis=1).copy().values[:, ind].sum()

    if ind_target_sum >= N_SPLITS:

        for seed in range(N_STARTS):

            skf = StratifiedKFold(n_splits = N_SPLITS, random_state = seed, shuffle = True)

            for n, (train_index, val_index) in enumerate(skf.split(fn_train, fn_targets[:,ind])):
                
                x_tr, x_val = fn_train[train_index], fn_train[val_index]
                y_tr, y_val = fn_targets[train_index,ind], fn_targets[val_index,ind]

                model = SVC(C = 40, cache_size = 2000)
                model.fit(x_tr, y_tr)
                svm0_test[:, ind] += model.decision_function(fn_test) / (N_SPLITS * N_STARTS)
                svm0_oof[val_index, ind] += model.decision_function(x_val) / N_STARTS

    score = log_loss(fn_targets[:, ind], svm0_oof[:, ind])
    print('SVM Target ind {} score {}:'.format(ind,score))

SVM Target ind 0 score 0.0267522871657328:
SVM Target ind 1 score 0.028325951116658196:
SVM Target ind 2 score 0.0377679348222106:
SVM Target ind 3 score 0.298996150675827:
SVM Target ind 4 score 0.47367284922854636:
SVM Target ind 5 score 0.11487746841755518:
SVM Target ind 6 score 0.0849778533499726:
SVM Target ind 7 score 0.15107173928883938:
SVM Target ind 8 score 0.0188839674111058:
SVM Target ind 9 score 0.41774788362703535:
SVM Target ind 10 score 0.566519022333145:
SVM Target ind 11 score 0.07872803928766862:
SVM Target ind 12 score 0.0110156476564788:
SVM Target ind 13 score 0.06609388593886781:
SVM Target ind 14 score 0.0188839674111058:
SVM Target ind 15 score 0.018883967411105797:
SVM Target ind 16 score 0.0755358696444202:
SVM Target ind 17 score 0.14005609163236157:
SVM Target ind 18 score 0.125893116074033:
SVM Target ind 19 score 0.0566519022333154:
SVM Target ind 20 score 0.058225566184240796:
SVM Target ind 21 score 0.1148774684175552:
SVM Target ind 22 score 0.009441

In [23]:
N_STARTS = 3
N_SPLITS = 5

svm1_test = np.zeros([len(fn_test),fn_targets.shape[1]])
svm1_oof = np.zeros([fn_targets.shape[0],fn_targets.shape[1]]) 

for ind in tqdm(range(len(target_feats))):

    ind_target_sum = targets.drop("sig_id", axis=1).copy().values[:, ind].sum()

    if ind_target_sum >= N_SPLITS:
        for seed in range(N_STARTS):
            skf = StratifiedKFold(n_splits = N_SPLITS, random_state = seed, shuffle = True)

            for n, (train_index, val_index) in enumerate(skf.split(svm0_oof, fn_targets[:,ind])):

                x_tr, x_val = svm0_oof[train_index, ind].reshape(-1, 1), svm0_oof[val_index, ind].reshape(-1, 1)
                y_tr, y_val = fn_targets[train_index,ind], fn_targets[val_index,ind]

                model = LogisticRegression(C = 35, max_iter = 1000)
                model.fit(x_tr, y_tr)
                svm1_test[:, ind] += model.predict_proba(svm0_test[:, ind].reshape(-1, 1))[:, 1] / (N_SPLITS * N_STARTS)
                svm1_oof[val_index, ind] += model.predict_proba(x_val)[:, 1] / N_STARTS

    score = log_loss(fn_targets[:, ind], svm0_oof[:, ind])
    print('SVM Target ind {} score {}:'.format(ind,score))

SVM Target ind 0 score 0.0267522871657328:
SVM Target ind 1 score 0.028325951116658196:
SVM Target ind 2 score 0.0377679348222106:
SVM Target ind 3 score 0.298996150675827:
SVM Target ind 4 score 0.47367284922854636:
SVM Target ind 5 score 0.11487746841755518:
SVM Target ind 6 score 0.0849778533499726:
SVM Target ind 7 score 0.15107173928883938:
SVM Target ind 8 score 0.0188839674111058:
SVM Target ind 9 score 0.41774788362703535:
SVM Target ind 10 score 0.566519022333145:
SVM Target ind 11 score 0.07872803928766862:
SVM Target ind 12 score 0.0110156476564788:
SVM Target ind 13 score 0.06609388593886781:
SVM Target ind 14 score 0.0188839674111058:
SVM Target ind 15 score 0.018883967411105797:
SVM Target ind 16 score 0.0755358696444202:
SVM Target ind 17 score 0.14005609163236157:
SVM Target ind 18 score 0.125893116074033:
SVM Target ind 19 score 0.0566519022333154:
SVM Target ind 20 score 0.058225566184240796:
SVM Target ind 21 score 0.1148774684175552:
SVM Target ind 22 score 0.009441

In [24]:
check_svm1 = np.zeros([targets.shape[0], targets.shape[1]-1])
check_svm1[cons_train_index,:] = svm1_oof
print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(check_svm1)))

OOF log loss:  0.015288382527672173


# submission

In [25]:
check = 0.2 * check_svm1 + 0.2 * check_xgb1 + 0.6 * check_tabnet1 #check_pytorch1
print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(check)))

OOF log loss:  0.015047465983673031


In [26]:
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

sub.loc[cons_test_index,target_feats] =  0.2 * svm1_test + 0.2 * xgb1_test + 0.6 * tabnet1_test #pytorch1_test 
sub.loc[noncons_test_index,target_feats] = 0
sub.to_csv('submission.csv', index=False)